In [2]:
import pandas as pd

In [3]:
air_reserve = pd.read_csv('air_reserve.csv')
air_visit_data = pd.read_csv('air_visit_data.csv')
air_visit_data = pd.read_csv('air_visit_data.csv')
date_info = pd.read_csv('date_info.csv')
hpg_store_info = pd.read_csv('hpg_store_info.csv')
store_id_relation = pd.read_csv('store_id_relation.csv')

In [4]:
#Split the file since its too big for github, extract the file outside of .git repository

hpg_reserve = pd.read_csv('C:\\Users\\mingt\\Downloads\\hpg_reserve.csv\\hpg_reserve.csv')
hpg_reserve.info()
df1 = hpg_reserve.iloc[0:100160]
df2 = hpg_reserve.iloc[100161:200320]
df1.to_csv('hpg_reserve1.csv')
df2.to_csv('hpg_reserve2.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000320 entries, 0 to 2000319
Data columns (total 4 columns):
hpg_store_id        object
visit_datetime      object
reserve_datetime    object
reserve_visitors    int64
dtypes: int64(1), object(3)
memory usage: 61.0+ MB


In [4]:
#concat files back after its been uploaded on github
hpg_reserve1 = pd.read_csv('hpg_reserve1.csv')
hpg_reserve2 = pd.read_csv('hpg_reserve2.csv')

hpg_reserve = pd.concat([hpg_reserve1, hpg_reserve2], axis=1)